## Load the data
## Clean the data
## Modelling

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('./data.csv')

In [4]:
X,y = data.drop('target',axis=1),data['target']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [7]:
import torch
import torch.nn as nn

In [8]:
import numpy as np

In [9]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [10]:
X_train.shape

torch.Size([227, 13])

In [11]:
X_test.shape

torch.Size([76, 13])

In [12]:
y_train.shape

torch.Size([227])

In [13]:
y_test.shape

torch.Size([76])

#### Modelling

In [14]:
import torch.nn.functional as F

In [15]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,512)
        self.fc5 = nn.Linear(512,1024)
        self.fc6 = nn.Linear(1024,512)
        self.fc7 = nn.Linear(512,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [16]:
device = torch.device('cuda')

In [17]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [18]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [19]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.view(len(preds),).to(device)
        y = y.view(len(y),).to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(preds,y):
    correct = 0
    total = 0
    for real,pred in zip(y_train,preds):
        if real == pred:
            correct += 1
        total += 1
    return round(correct/total,3)

In [20]:
import wandb

In [21]:
from tqdm import tqdm

In [22]:
EPOCHS = 212
# EPOCHS = 100

In [23]:
# model = Test_Model().to(device)
# optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
# criterion = nn.L1Loss()
# wandb.init(project=PROJECT_NAME,name='baseline')
# for _ in tqdm(range(EPOCHS)):
#     preds = model(X_train.float().to(device))
#     preds = preds.view(len(preds),)
#     preds.to(device)
#     loss = criterion(preds,y_train)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
# wandb.finish()

In [24]:
# preds[:10]

In [25]:
# preds = torch.round(preds)

In [26]:
# correct = 0
# total = 0
# for real,pred in zip(y_train,preds):
#     if real == pred:
#         correct += 1
# #     total += 1

In [27]:
# round(correct/total,3)

In [28]:
## Testing Modelling

In [29]:
import torch
import torch.nn as nn

In [30]:
class Test_Model(nn.Module):
    def __init__(self,num_of_layers=1,activation=F.relu,input_shape=13,fc1_output=32,fc2_output=64,fc3_output=128,fc4_output=256,output_shape=1):
        super().__init__()
        self.num_of_layers = num_of_layers
        self.activation = activation
        self.fc1 = nn.Linear(input_shape,fc1_output)
        self.fc2 = nn.Linear(fc1_output,fc2_output)
        self.fc3 = nn.Linear(fc2_output,fc3_output)
        self.fc4 = nn.Linear(fc3_output,fc4_output)
        self.fc5 = nn.Linear(fc4_output,fc3_output)
        self.fc6 = nn.Linear(fc3_output,fc3_output)
        self.fc7 = nn.Linear(fc3_output,output_shape)
    
    def forward(self,X,activation=False):
        preds = self.fc1(X)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc2(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc3(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc4(preds)
        if activation:
                 preds = self.activation(preds)
        preds = self.fc5(preds)
        if activation:
                 preds = self.activation(preds)
        for _ in range(self.num_of_layers):
            preds = self.fc6(preds)
            if activation:
                     preds = self.activation(preds)
        preds =  self.fc7(preds)
        preds = F.sigmoid(preds)
        return preds

In [31]:
device = torch.device('cuda')

In [32]:
# preds = torch.round(preds)

In [33]:
# num_of_layers = 1
# fc1_output
# fc2_output
# fc3_output
# fc4_output
# optimizer = torch.optim.SGD
# criterion = nn.MSELoss
# lr = 0.125
# activtion = nn.Tanh()

In [39]:
fc1_outputs = [16,32,64,128,1]
for fc1_output in fc1_outputs:
    model = Test_Model(num_of_layers=1,activation=nn.Tanh()fc1_outputs=fc1_output).to(device)
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(),lr=0.125)
    criterion = nn.MSELoss()
    wandb.init(project=PROJECT_NAME,name=f'fc1_output-{fc1_output}')
    for _ in tqdm(range(212)):
        preds = model(X_train.float().to(device),True)
        preds = preds.view(len(preds),)
        preds.to(device)
        loss = criterion(preds,y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(preds,y_train)})
    wandb.finish()

100%|██████████| 212/212 [00:01<00:00, 165.70it/s]


loss,0.23292
val_loss,0.2686
accuracy,0.0
_runtime,8
_timestamp,1621483095
_step,211


loss,███████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▄▄▃▃▃▃▁▃▄▃▂▃▃▅▅▂▃
val_loss,▂▃▄▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▆█▄▃▄▃▃▄▁▃
accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


100%|██████████| 212/212 [00:01<00:00, 162.48it/s]


loss,0.24875
val_loss,0.32447
accuracy,0.0
_runtime,8
_timestamp,1621483114
_step,211


...

loss,0.25052
val_loss,0.39434
accuracy,0.0
_runtime,8
_timestamp,1621483235
_step,211


loss,████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
val_loss,████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁
accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████████████
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████████████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


100%|██████████| 212/212 [00:01<00:00, 146.93it/s]


loss,0.25017
val_loss,0.2486
accuracy,0.0
_runtime,8
_timestamp,1621483255
_step,211


loss,█████▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
val_loss,████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████████████
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████████████████████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
